In [1]:
import pandas as pd
import numpy as np

In [128]:
train_data_file ='SpamData/02_Training/train_data.txt'
test_data_file ='SpamData/02_Training/test_data.txt'
vocab_size = 2500
token_spam_prob_file ='SpamData/03_Testing/prob-spam.txt'
token_ham_prob_file ='SpamData/03_Testing/prob-nonspam.txt'
token_all_prob_file ='SpamData/03_Testing/prob-all-tokens.txt'
test_feature_matrix_01 = 'SpamData/03_Testing/test-feature-01.txt'
test_feature_matrix_02 = 'SpamData/03_Testing/test-feature-02.txt'
test_feature_matrix_03 = 'SpamData/03_Testing/test-feature-03.txt'
test_feature_matrix_04 = 'SpamData/03_Testing/test-feature-04.txt'
test_feature_matrix_05 = 'SpamData/03_Testing/test-feature-05.txt'
test_feature_matrix_06 = 'SpamData/03_Testing/test-feature-06.txt'
test_feature_matrix_07 = 'SpamData/03_Testing/test-feature-07.txt'
test_feature_matrix_08 = 'SpamData/03_Testing/test-feature-08.txt'
test_target_file = 'SpamData/03_Testing/test-target.txt'

In [3]:
#Read and Load features from .txt files into numpy array
sparse_train_data = np.loadtxt(train_data_file,delimiter=' ',dtype=int)
sparse_test_data = np.loadtxt(test_data_file,delimiter=' ',dtype=int)

In [4]:
sparse_train_data[:5] #first 5 rows

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [5]:
sparse_train_data[-5:] #last 5 rows

array([[5798, 2035,    0,    2],
       [5798, 2040,    0,    1],
       [5798, 2272,    0,    1],
       [5798, 2342,    0,    1],
       [5798, 2370,    0,    1]])

In [6]:
print(sparse_train_data.shape[0],'==> No of rows in TRAIN file')
print(sparse_test_data.shape[0],'==> No of rows in TEST file')

263535 ==> No of rows in TRAIN file
112601 ==> No of rows in TEST file


In [7]:
print(np.unique(sparse_train_data[:,0]).size,'==> No of emails in TRAIN file')
print(np.unique(sparse_test_data[:,0]).size,'==> No of emails in TEST file')
#[:,0] ==> Notation for selecting all the rows in the 0th column.

4012 ==> No of emails in TRAIN file
1725 ==> No of emails in TEST file


# Creating a full Matrix

In [8]:
column_names=['DOC_ID']+['CATEGORY']+list(range(0,vocab_size))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [9]:
len(column_names)

2502

In [10]:
index_names = np.unique(sparse_train_data[:,0])
index_names

array([   0,    1,    2, ..., 5796, 5797, 5798])

In [11]:
full_train_data = pd.DataFrame(index=index_names,columns=column_names)
full_train_data.fillna(value=0,inplace=True)
#fillna fills the na values with 0s

In [12]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Creating a Full Matrix from Sparse Matrix

In [13]:
def make_full_matrix(sparse_matrix,no_words,doc_idx=0,word_idx=1,cat_idx=2,freq_idx=3):
    #no_words - size of vocabulary
    #doc_idx=0 - position of document id in sparse matrix set to 1st column
    #word_idx=1 - position of word id in sparse matrix set to 2nd column 
    #cat_idx=2 - position of label/category (spam -> 1 , non-spam -> 0) set to 3rd column
    #freq_idx=3 - position of occurrence of word  id in sparse matrix set to 1st column
    column_names = column_names=['DOC_ID']+['CATEGORY']+list(range(0,vocab_size))
    doc_id_names = np.unique(sparse_matrix[:,0])
    full_matrix = pd.DataFrame(index=index_names,columns=column_names)
    full_matrix.fillna(value=0,inplace=True)
    for i in range(sparse_matrix.shape[0]):
        doc_no = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_no,'DOC_ID'] = doc_no
        full_matrix.at[doc_no,'CATEGORY'] = label
        full_matrix.at[doc_no,word_id] = occurence
    full_matrix.set_index('DOC_ID',inplace=True)
    return full_matrix

In [14]:
%%time
full_train_data = make_full_matrix(sparse_train_data,vocab_size)

CPU times: total: 31.5 s
Wall time: 31.9 s


In [15]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


 # Counting the tokens to train the Naive Bayes Model


In [16]:
#calculating token Probabilities
# P(Spam | Viagra) =   ( P(Viagra | Spam) * P(Spam) ) / P(Viagra)

### Calculating the probability of spam - the percent of spam messages in the training dataset & storing it to var prob_spam

In [17]:
full_train_data.CATEGORY.size

4012

In [18]:
full_train_data.CATEGORY.sum()

1246

In [19]:
full_train_data.CATEGORY.tail()

DOC_ID
5792    0
5795    0
5796    0
5797    0
5798    0
Name: CATEGORY, dtype: int64

In [20]:
prob_spam = full_train_data.CATEGORY.sum()/full_train_data.CATEGORY.size

In [21]:
print('probability of spam is', prob_spam)

probability of spam is 0.31056829511465606


### Total Number of Words/Tokens

In [22]:
full_train_features = full_train_data.loc[:,full_train_data.columns !='CATEGORY']

In [23]:
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4012,)

In [25]:
email_lengths[0:5]

DOC_ID
0     87
1     53
2     40
3    183
4     43
dtype: int64

In [26]:
total_wc = email_lengths.sum()
total_wc

447628

### Number of token in Spam & Ham Emails

In [27]:
#Creating a subset of the email_lengths series that only contains the spam messages.
spam_lengths = email_lengths[full_train_data.CATEGORY==1]
spam_lengths.shape

(1246,)

In [28]:
ham_lengths = email_lengths [full_train_data.CATEGORY==0]
ham_lengths.shape

(2766,)

In [29]:
email_lengths.shape[0]-spam_lengths.shape[0]-ham_lengths.shape[0]
#Verifying

0

In [30]:
nonspam_wc = ham_lengths.sum()
nonspam_wc

254877

In [31]:
spam_wc = spam_lengths.sum()
spam_wc

192751

In [32]:
spam_wc + nonspam_wc - total_wc

0

In [33]:
print("average number of words in spam emails  {:.2f}".format(spam_wc/spam_lengths.shape[0]))
print("average number of words in ham emails  {:.2f}".format(nonspam_wc/ham_lengths.shape[0]))

average number of words in spam emails  154.70
average number of words in ham emails  92.15


### Summing the tokens Occuring in Spam

In [34]:
full_train_features.shape

(4012, 2500)

In [35]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
train_spam_tokens.shape

(1246, 2500)

In [37]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1
summed_spam_tokens.shape

(2500,)

In [38]:
summed_spam_tokens.tail()

2495     4
2496     2
2497    17
2498    18
2499     2
dtype: int64

In [39]:
#Repeating same process for ham messages..  and storing them in summed_ham_tokens

### Summing the tokens Occuring in ham

In [40]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY==0]
summed_ham_tokens = train_ham_tokens.sum(axis=0)+1

In [41]:
summed_ham_tokens.tail()

2495    33
2496    34
2497    17
2498    12
2499    31
dtype: int64

In [42]:
train_ham_tokens[2499].sum()+1

31

# Calculating the token probabilities and Saving the trained model

In [43]:
# P(Token|Spam) - probability that token occurs given the email is spam
prob_tokens_spam = summed_spam_tokens/(spam_wc + vocab_size)
#we add vocab_size because of the smothing... as we added +1 before..
prob_tokens_spam[:5]

0    0.011334
1    0.005173
2    0.006996
3    0.010494
4    0.007068
dtype: float64

In [44]:
prob_tokens_spam.sum()

1.0

In [45]:
#P(Token|Ham) - probability that a tokeb occurs given the email is nonspam
prob_tokens_nonspam = summed_ham_tokens/(nonspam_wc + vocab_size)
prob_tokens_nonspam[:5]

0    0.021008
1    0.010386
2    0.008031
3    0.003532
4    0.006457
dtype: float64

In [46]:
prob_tokens_nonspam.sum()

1.0

In [47]:
# P(Token) - probability that token occurs
prob_tokens_all =full_train_features.sum(axis=0)/total_wc
prob_tokens_all.sum()

1.0

In [48]:
#Saving the Trained Model
np.savetxt(token_spam_prob_file,prob_tokens_spam)
np.savetxt(token_ham_prob_file,prob_tokens_nonspam)
np.savetxt(token_all_prob_file,prob_tokens_all)

In [49]:
#we use make_full_matrix to prepare test data,
# by calling it and running test data through it saving it as a file for a later use.

# Preparing Test Data

In [50]:
sparse_test_data.shape

(112601, 4)

In [51]:
%%time
full_test_data = make_full_matrix(sparse_test_data,no_words=vocab_size)

CPU times: total: 5min 17s
Wall time: 5min 22s


In [105]:
x_test = full_test_data.loc[:,full_test_data.columns !='CATEGORY']
y_test = full_test_data.CATEGORY

In [107]:
# x_test 2499 + DOC_ID
# len(x_test.columns) 2500
# x_test_split = np.array_split(x_test,5)
#Checking if the split is good
# pd.DataFrame(x_test_split[0])[100:102]
# columns=(np.append(['DOC_ID'],list(range(len(x_test.columns)-1))))

In [129]:
def splitter(df,size=0.5):
    total_rows = len(df)
    test_rows = int(total_rows * size)
    train_rows = total_rows - test_rows
    split_indices = [train_rows, total_rows]
    # Split the DataFrame based on the split indices
    return(df.iloc[:split_indices[0]], df.iloc[split_indices[0]:])

In [130]:
splitted_x1,splitted_x2 = splitter(x_test)
half_01,half_02 = splitter(splitted_x1)
half_03,half_04 = splitter(splitted_x2)
x_test_01,x_test_02 = splitter(half_01)
x_test_03,x_test_04 = splitter(half_02)
x_test_05,x_test_06 = splitter(half_03)
x_test_07,x_test_08 = splitter(half_04)

In [131]:
print(x_test_01.shape)
print(x_test_02.shape)
print(x_test_03.shape)
print(x_test_04.shape)
print(x_test_05.shape)
print(x_test_06.shape)
print(x_test_07.shape)
print(x_test_08.shape)

(718, 2500)
(717, 2500)
(717, 2500)
(717, 2500)
(717, 2500)
(717, 2500)
(717, 2500)
(717, 2500)


In [132]:
#Saving y_test and 5 splits of x_test
np.savetxt(test_target_file,y_test)
np.savetxt(test_feature_matrix_01,x_test_01)
np.savetxt(test_feature_matrix_02,x_test_02)
np.savetxt(test_feature_matrix_03,x_test_03)
np.savetxt(test_feature_matrix_04,x_test_04)
np.savetxt(test_feature_matrix_05,x_test_05)
np.savetxt(test_feature_matrix_06,x_test_06)
np.savetxt(test_feature_matrix_07,x_test_07)
np.savetxt(test_feature_matrix_08,x_test_08)